In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
def morgan(x):
    return np.array(AllChem.GetMorganFingerprintAsBitVect(x,2,nBits = 2048))

import pandas as pd
import numpy as np
import os

from moses.metrics.utils import get_mol
from sklearn.decomposition import PCA


load data

In [2]:
df = pd.read_csv('../data/data_sim_score_30k_score.csv')

In [10]:
df['mol'] = df['Smiles'].apply(get_mol)
df['mfp'] = df['mol'].apply(morgan)

In [5]:
from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso, BayesianRidge, ARDRegression, LassoCV
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor,BaggingRegressor
import sklearn.metrics as sm
from numpy import mean,std

from sklearn.kernel_ridge import KernelRidge

comparison

In [75]:
import timeit

def eva_model(data_, spl_c, train_l, test_l, feature_c, label_c, model_):
        train_set = np.vstack(data_[[x in train_l for x in data_[spl_c]]][feature_c])
        train_label = data_[[x in train_l for x in data_[spl_c]]][label_c]
        test_set =np.vstack(data_[[x in test_l for x in data_[spl_c]]][feature_c])
        test_label =data_[[x in test_l for x in data_[spl_c]]][label_c]
        #train model
        clf_ = model_()
        start_ = timeit.default_timer()
        clf_.fit(train_set, train_label)
        y_predict = clf_.predict(test_set)
        end_ = timeit.default_timer()
        time_ = round(end_-start_,2)
        #evaluation
        mae_= sm.mean_absolute_error(test_label, y_predict)
        rmse_= np.sqrt(sm.mean_squared_error(test_label, y_predict))
        r2_= sm.r2_score(test_label, y_predict)
        corr_= np.corrcoef(test_label, y_predict)[0][1]
        
        print("MAE得分:", mae_)
        print("RMSE得分:", rmse_)
        print("R2得分:", r2_)
        print("Corr得分:", corr_)
        print('predicting time: %s seconds'%(time_),' for',len(y_predict),'samples.')
        
        return clf_, y_predict, mae_, rmse_, r2_,corr_,time_
    
    


def eva_model_pca(data_, spl_c, train_l, test_l, feature_c, label_c, model_, pca_trans):
        train_set = pca_trans.transform(np.vstack(data_[[x in train_l for x in data_[spl_c]]][feature_c]))
        train_label = data_[[x in train_l for x in data_[spl_c]]][label_c]
        test_set =pca_trans.transform(np.vstack(data_[[x in test_l for x in data_[spl_c]]][feature_c]))
        test_label =data_[[x in test_l for x in data_[spl_c]]][label_c]
        #train model
        clf_ = model_()
        start_ = timeit.default_timer()
        clf_.fit(train_set, train_label)
        y_predict = clf_.predict(test_set)
        end_ = timeit.default_timer()
        time_ = round(end_-start_,2)
        #evaluation
        mae_= sm.mean_absolute_error(test_label, y_predict)
        rmse_= np.sqrt(sm.mean_squared_error(test_label, y_predict))
        r2_= sm.r2_score(test_label, y_predict)
        corr_= np.corrcoef(test_label, y_predict)[0][1]
        
        print("MAE得分:", mae_)
        print("RMSE得分:", rmse_)
        print("R2得分:", r2_)
        print("Corr得分:", corr_)
        print('running time: %s seconds'%(time_),' for',len(y_predict),'samples.')

        return clf_, y_predict, mae_, rmse_, r2_,corr_,time_

In [39]:
train_set = np.vstack(df_new[[x in ['inh','com','0'] for x in df_new['type']]]['mfp'])
train_label = df_new[[x in ['inh','com','0'] for x in df_new['type']]]['score']

pca_ = PCA(n_components=100).fit(train_set)

In [76]:
res_rl = pd.DataFrame()
best_r2 = 0

for m in [BayesianRidge, 
          ARDRegression, LinearRegression,SGDRegressor,Ridge, LassoCV,MLPRegressor,
          SVR, KernelRidge,
          KNeighborsRegressor,DecisionTreeRegressor,
          RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor,BaggingRegressor
         ]:
    print(m())
    lmae = []
    lrmse = []
    lr2 = []
    lcorr = []
    ltime = []
    l_clf = []

    if m == SVR:
        clf_,y_,mae_,rmse_,r2_,corr_,time_ = eva_model_pca(df_new, 'type', ['inh','com','0'], ['rl'], 'mfp', 'score', m, pca_)
    else:
        clf_,y_,mae_,rmse_,r2_,corr_,time_ = eva_model(df_new, 'type', ['inh','com','0'], ['rl'], 'mfp', 'score', m)
    lmae.append(mae_)
    lrmse.append(rmse_)
    lr2.append(r2_)
    lcorr.append(corr_)
    ltime.append(time_)
    l_clf.append(clf_)
    res_ = pd.DataFrame({'clf':l_clf,'mae':lmae,'rmse':lrmse,'r2':lr2,'corr':lcorr,'time':ltime},index = [str(m())])
    res_rl = pd.concat([res_rl,res_])
    print()

BayesianRidge()
MAE得分: 0.418580134575959
RMSE得分: 0.52358503895206
R2得分: 0.1305215195123023
Corr得分: 0.5386551672132204
predicting time: 2.15 seconds  for 4470 samples.

ARDRegression()
MAE得分: 0.4389607783024685
RMSE得分: 0.5509675943768678
R2得分: 0.03719908729484056
Corr得分: 0.508005372987357
predicting time: 73.26 seconds  for 4470 samples.

LinearRegression()
MAE得分: 0.4537271908628632
RMSE得分: 0.5759890825949522
R2得分: -0.05223534175777478
Corr得分: 0.5005711714152704
predicting time: 1.16 seconds  for 4470 samples.

SGDRegressor()
MAE得分: 0.42319521523488257
RMSE得分: 0.5327863279784608
R2得分: 0.0996932125452541
Corr得分: 0.514229507312013
predicting time: 0.75 seconds  for 4470 samples.

Ridge()
MAE得分: 0.4472435050092456
RMSE得分: 0.5655338732647911
R2得分: -0.014382214664683035
Corr得分: 0.5113087662505703
predicting time: 0.24 seconds  for 4470 samples.

LassoCV()
MAE得分: 0.4301596877863613
RMSE得分: 0.5362585575362858
R2得分: 0.08792016898903687
Corr得分: 0.5232676044616938
predicting time: 27.01 seconds  

In [77]:
res_rl.sort_values('corr', ascending = False)

,clf,mae,rmse,r2,corr,time
BayesianRidge(),BayesianRidge(),0.418580,0.523585,0.130522,0.538655,2.15
LassoCV(),LassoCV(),0.430160,0.536259,0.087920,0.523268,27.01
SGDRegressor(),SGDRegressor(),0.423195,0.532786,0.099693,0.514230,0.75
Ridge(),Ridge(),0.447244,0.565534,-0.014382,0.511309,0.24
ARDRegression(),ARDRegression(),0.438961,0.550968,0.037199,0.508005,73.26
LinearRegression(),LinearRegression(),0.453727,0.575989,-0.052235,0.500571,1.16
KernelRidge(),KernelRidge(),0.438816,0.564870,-0.012002,0.490045,2.09
SVR(),SVR(),0.405321,0.515393,0.157516,0.481910,2.92
GradientBoostingRegressor(),([DecisionTreeRegressor(criterion='friedman_ms...,0.435946,0.554042,0.026426,0.399464,30.56
MLPRegressor(),MLPRegressor(),0.634703,0.810119,-1.081526,0.317163,18.21


In [62]:
from sklearn.model_selection import KFold
df_rl = df_new[[x in ['rl'] for x in df_new['type']]]
df_rl.reset_index(drop = True, inplace = True)

res_cv = pd.DataFrame()
res_l = []
for j in range(10):
    df_test = df_rl.sample(500)
    test_set_all =np.vstack(df_test['mfp'])
    test_set_pca =pca_.transform(np.vstack(df_test['mfp']))
    test_label =df_test['score']
    res_500 = pd.DataFrame()

    for i in res_rl['clf']:
        if str(i) == 'SVR()':
            test_set =test_set_pca    
        else:
            test_set =test_set_all
        start_ = timeit.default_timer()
        y_predict = i.predict(test_set)
        end_ = timeit.default_timer()
        time_ = end_-start_
        time_log = round(np.log(end_-start_),2)
        #evaluation
        mae_= sm.mean_absolute_error(test_label, y_predict)
        rmse_= np.sqrt(sm.mean_squared_error(test_label, y_predict))
        r2_= sm.r2_score(test_label, y_predict)
        corr_= np.corrcoef(test_label, y_predict)[0][1]
        print(str(i))
        print("MAE得分:", mae_)
        print("RMSE得分:", rmse_)
        print("R2得分:", r2_)
        print("Corr得分:", corr_)
        print('predicting time: %s seconds'%(time_),' for',len(y_predict),'samples.')
        res_ = pd.DataFrame({
            'clf':[str(i)[:-2]],
            'r2':[r2_],'corr':[corr_],'mae':[mae_],'rmse':[rmse_],
            'time':[time_],'time_log':[time_log]},index = [str(i)[:-2]])
        res_500 = pd.concat([res_500,res_])
    
    print()
    res_cv = pd.concat([res_cv,res_500])

/home/wu_junyu/anaconda3/envs/rdk/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/wu_junyu/anaconda3/envs/rdk/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


BayesianRidge()
MAE得分: 0.414114764312465
RMSE得分: 0.5162299401489168
R2得分: 0.14158075494930933
Corr得分: 0.5515031946194828
predicting time: 0.0011106189340353012 seconds  for 500 samples.
ARDRegression()
MAE得分: 0.4312089265160156
RMSE得分: 0.5379958089822587
R2得分: 0.06766743756825722
Corr得分: 0.5307816618212744
predicting time: 0.0008423104882240295 seconds  for 500 samples.
LinearRegression()
MAE得分: 0.4536396245292798
RMSE得分: 0.5731599507743707
R2得分: -0.05819265068667767
Corr得分: 0.5039162788219499
predicting time: 0.0006911121308803558 seconds  for 500 samples.
SGDRegressor()
MAE得分: 0.396083127774126
RMSE得分: 0.4966535769790804
R2得分: 0.20545188272490356
Corr得分: 0.5489917632446715
predicting time: 0.0006696134805679321 seconds  for 500 samples.
Ridge()
MAE得分: 0.4453503049167066
RMSE得分: 0.5616647253130396
R2得分: -0.016172332953207746
Corr得分: 0.5155326152917934
predicting time: 0.0006584282964468002 seconds  for 500 samples.
LassoCV()
MAE得分: 0.42227581687364873
RMSE得分: 0.5251148122901055
R2得分: 

BaggingRegressor()
MAE得分: 0.523184354
RMSE得分: 0.6525316151419316
R2得分: -0.2900979660674836
Corr得分: 0.3620873494384771
predicting time: 0.02781093120574951 seconds  for 500 samples.

BayesianRidge()
MAE得分: 0.41713121652813206
RMSE得分: 0.5066256901809418
R2得分: 0.12572919359501844
Corr得分: 0.5956698175623543
predicting time: 0.001108771190047264 seconds  for 500 samples.
ARDRegression()
MAE得分: 0.43936682808564526
RMSE得分: 0.5383211907730568
R2得分: 0.012915091995956196
Corr得分: 0.5480665597738494
predicting time: 0.000806247815489769 seconds  for 500 samples.
LinearRegression()
MAE得分: 0.4552368630152602
RMSE得分: 0.5610917466830838
R2得分: -0.07235682698917989
Corr得分: 0.5394075419782964
predicting time: 0.0006679166108369827 seconds  for 500 samples.
SGDRegressor()
MAE得分: 0.4051800351679609
RMSE得分: 0.49777410229607577
R2得分: 0.1560122259053489
Corr得分: 0.5682457775344241
predicting time: 0.0006506089121103287 seconds  for 500 samples.
Ridge()
MAE得分: 0.44990487964486353
RMSE得分: 0.5510999241359567
R2得分

BaggingRegressor()
MAE得分: 0.521728224
RMSE得分: 0.6513344645170651
R2得分: -0.22161524787952502
Corr得分: 0.35687361544006696
predicting time: 0.02769484557211399 seconds  for 500 samples.

BayesianRidge()
MAE得分: 0.4220125709695425
RMSE得分: 0.5211659339935806
R2得分: 0.18844394831739375
Corr得分: 0.5554771293469565
predicting time: 0.001207863911986351 seconds  for 500 samples.
ARDRegression()
MAE得分: 0.4345170148595092
RMSE得分: 0.5442469904046482
R2得分: 0.1149688589048612
Corr得分: 0.5238162902663096
predicting time: 0.0009586364030838013 seconds  for 500 samples.
LinearRegression()
MAE得分: 0.45316227026187395
RMSE得分: 0.5745215273638881
R2得分: 0.013767993530017564
Corr得分: 0.49559930491386567
predicting time: 0.0009297560900449753 seconds  for 500 samples.
SGDRegressor()
MAE得分: 0.40501626994738965
RMSE得分: 0.5041459366755504
R2得分: 0.24058526006668735
Corr得分: 0.5533058963528945
predicting time: 0.0008432697504758835 seconds  for 500 samples.
Ridge()
MAE得分: 0.4499515025153787
RMSE得分: 0.5658439925884674
R2得

BaggingRegressor()
MAE得分: 0.541767374
RMSE得分: 0.6617030719349171
R2得分: -0.5331631023305861
Corr得分: 0.29069593320656595
predicting time: 0.027494149282574654 seconds  for 500 samples.

BayesianRidge()
MAE得分: 0.4273448590448086
RMSE得分: 0.5238632154910347
R2得分: 0.07040699209673551
Corr得分: 0.566709230027788
predicting time: 0.001127980649471283 seconds  for 500 samples.
ARDRegression()
MAE得分: 0.4664643890808173
RMSE得分: 0.5759016427107798
R2得分: -0.12344980817379025
Corr得分: 0.5052437291820729
predicting time: 0.0008020959794521332 seconds  for 500 samples.
LinearRegression()
MAE得分: 0.4669858848401675
RMSE得分: 0.5817611522213041
R2得分: -0.1464271814403859
Corr得分: 0.5272966803198089
predicting time: 0.0006606001406908035 seconds  for 500 samples.
SGDRegressor()
MAE得分: 0.40129983020209187
RMSE得分: 0.49858999811235194
R2得分: 0.1579378160979611
Corr得分: 0.5587511406205479
predicting time: 0.0006572268903255463 seconds  for 500 samples.
Ridge()
MAE得分: 0.460872861664254
RMSE得分: 0.5711143687842993
R2得分: 

In [85]:
res_cv.groupby(
    'clf'
).agg(
    ['mean','std']
).sort_values(
    ('r2','mean'), ascending = False
)

r2                corr                 mae  \
                               mean       std      mean       std      mean   
clf                                                                           
BayesianRidge              0.191038  0.048403  0.558283  0.025313  0.417932   
SGDRegressor               0.189147  0.042679  0.545132  0.026155  0.400543   
SVR                        0.169216  0.046951  0.496919  0.030040  0.404142   
LassoCV                    0.099719  0.064482  0.546852  0.029897  0.427686   
ARDRegression              0.034198  0.081518  0.521031  0.028345  0.439155   
GradientBoostingRegressor  0.028345  0.065632  0.406363  0.036464  0.438599   
Ridge                     -0.024408  0.077675  0.520969  0.021034  0.451589   
KernelRidge               -0.032981  0.077220  0.491353  0.022490  0.442657   
AdaBoostRegressor         -0.045199  0.045314  0.261413  0.043909  0.458373   
LinearRegression          -0.065533  0.085305  0.509095  0.020724  0.458579   
RandomForestRegressor     -0.364718  0.086761  0.317458  0.034627  0.528006   
BaggingRegressor          -0.366428  0.104571  0.328229  0.035275  0.523926   
MLPRegressor              -1.187736  0.229086  0.398321  0.027028  0.652464   
DecisionTreeRegressor     -1.565654  0.215250  0.213146  0.030522  0.694758   
ExtraTreesRegressor       -1.596158  0.192757  0.232932  0.026219  0.710564   
KNeighborsRegressor       -3.122407  0.245955  0.296295  0.031448  0.955554   

                                         rmse                time            \
                                std      mean       std      mean       std   
clf                                                                           
BayesianRidge              0.011266  0.518238  0.013131  0.001130  0.000030   
SGDRegressor               0.011049  0.503377  0.013315  0.000691  0.000060   
SVR                        0.013738  0.509572  0.016997  0.155149  0.000909   
LassoCV                    0.013533  0.530144  0.014395  0.000663  0.000030   
ARDRegression              0.015956  0.548955  0.018358  0.000838  0.000046   
GradientBoostingRegressor  0.015470  0.550960  0.020104  0.002023  0.000018   
Ridge                      0.012355  0.565371  0.011920  0.000670  0.000034   
KernelRidge                0.012899  0.567727  0.011896  0.059893  0.000073   
AdaBoostRegressor          0.019215  0.571920  0.024522  0.042420  0.000070   
LinearRegression           0.013240  0.576542  0.012872  0.000711  0.000080   
RandomForestRegressor      0.014957  0.652855  0.016934  0.012703  0.000113   
BaggingRegressor           0.015924  0.652993  0.016927  0.027675  0.000460   
MLPRegressor               0.018688  0.825377  0.024755  0.001712  0.000088   
DecisionTreeRegressor      0.021813  0.894539  0.023572  0.001262  0.000012   
ExtraTreesRegressor        0.026393  0.900351  0.030789  0.012211  0.000163   
KNeighborsRegressor        0.021965  1.134686  0.025042  0.101516  0.000309   

                          time_log            
                              mean       std  
clf                                           
BayesianRidge               -6.786  0.025033  
SGDRegressor                -7.281  0.081165  
SVR                         -1.862  0.006325  
LassoCV                     -7.319  0.042804  
ARDRegression               -7.085  0.052546  
GradientBoostingRegressor   -6.204  0.009661  
Ridge                       -7.310  0.050111  
KernelRidge                 -2.815  0.005270  
AdaBoostRegressor           -3.160  0.000000  
LinearRegression            -7.253  0.101440  
RandomForestRegressor       -4.366  0.009661  
BaggingRegressor            -3.587  0.014944  
MLPRegressor                -6.370  0.049889  
DecisionTreeRegressor       -6.674  0.009661  
ExtraTreesRegressor         -4.407  0.014181  
KNeighborsRegressor         -2.289  0.003162

training and save the BaysianRidge regressor as reward function.

In [89]:
train_set = np.vstack(df_new['mfp'])
train_label = df_new['score']

#train model
clf_ = BayesianRidge()
clf_.fit(train_set, train_label)

BayesianRidge()

In [100]:
import joblib
joblib.dump(clf_, '../reward_function/Bayesian_Regressor.pkl')

['../reward_function/Bayesian_Regressor.pkl']